# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 10 2023 8:30AM,254615,10,0086286484,ISDIN Corporation,Released
1,Jan 10 2023 8:30AM,254615,10,0086286488,ISDIN Corporation,Released
2,Jan 10 2023 8:30AM,254615,10,0086286495,ISDIN Corporation,Released
3,Jan 10 2023 8:30AM,254615,10,0086286497,ISDIN Corporation,Released
4,Jan 10 2023 8:30AM,254615,10,0086286509,ISDIN Corporation,Released
5,Jan 10 2023 8:30AM,254615,10,0086286510,ISDIN Corporation,Released
6,Jan 10 2023 8:30AM,254615,10,0086286512,ISDIN Corporation,Released
7,Jan 10 2023 8:30AM,254596,10,SOTAH0000883,"Nextsource Biotechnology, LLC",Released
8,Jan 10 2023 8:30AM,254596,10,SONSB0002074,"Nextsource Biotechnology, LLC",Released
9,Jan 10 2023 8:30AM,254596,10,SOTAH0000884,"Nextsource Biotechnology, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,254612,Released,1
29,254613,Released,6
30,254615,Released,7
31,254616,Released,1
32,254617,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254612,NaN,NaN,1.0
254613,NaN,NaN,6.0
254615,NaN,NaN,7.0
254616,NaN,NaN,1.0
254617,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254516,6.0,2.0,1.0
254533,2.0,0.0,1.0
254558,0.0,0.0,1.0
254562,0.0,0.0,1.0
254563,23.0,16.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254516,6,2,1
254533,2,0,1
254558,0,0,1
254562,0,0,1
254563,23,16,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254516,6,2,1
1,254533,2,0,1
2,254558,0,0,1
3,254562,0,0,1
4,254563,23,16,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254516,6,2,1
1,254533,2,,1
2,254558,,,1
3,254562,,,1
4,254563,23,16,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation
7,Jan 10 2023 8:30AM,254596,10,"Nextsource Biotechnology, LLC"
10,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc."
13,Jan 9 2023 4:27PM,254616,15,Lonza
14,Jan 9 2023 4:04PM,254613,15,"Person & Covey, Inc."
20,Jan 9 2023 4:00PM,254612,19,"GUSA Granules USA, Inc."
21,Jan 9 2023 3:52PM,254610,15,"Virtus Pharmaceuticals, LLC"
30,Jan 9 2023 3:51PM,254608,15,"Alliance Pharma, Inc."
46,Jan 9 2023 3:51PM,254609,19,"GUSA Granules USA, Inc."
47,Jan 9 2023 3:47PM,254607,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation,,,7
1,Jan 10 2023 8:30AM,254596,10,"Nextsource Biotechnology, LLC",,,3
2,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc.",,,3
3,Jan 9 2023 4:27PM,254616,15,Lonza,,,1
4,Jan 9 2023 4:04PM,254613,15,"Person & Covey, Inc.",,,6
5,Jan 9 2023 4:00PM,254612,19,"GUSA Granules USA, Inc.",,,1
6,Jan 9 2023 3:52PM,254610,15,"Virtus Pharmaceuticals, LLC",,,9
7,Jan 9 2023 3:51PM,254608,15,"Alliance Pharma, Inc.",,,16
8,Jan 9 2023 3:51PM,254609,19,"GUSA Granules USA, Inc.",,,1
9,Jan 9 2023 3:47PM,254607,15,"Brookfield Pharmaceuticals, LLC",,,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation,7,,
1,Jan 10 2023 8:30AM,254596,10,"Nextsource Biotechnology, LLC",3,,
2,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc.",3,,
3,Jan 9 2023 4:27PM,254616,15,Lonza,1,,
4,Jan 9 2023 4:04PM,254613,15,"Person & Covey, Inc.",6,,
5,Jan 9 2023 4:00PM,254612,19,"GUSA Granules USA, Inc.",1,,
6,Jan 9 2023 3:52PM,254610,15,"Virtus Pharmaceuticals, LLC",9,,
7,Jan 9 2023 3:51PM,254608,15,"Alliance Pharma, Inc.",16,,
8,Jan 9 2023 3:51PM,254609,19,"GUSA Granules USA, Inc.",1,,
9,Jan 9 2023 3:47PM,254607,15,"Brookfield Pharmaceuticals, LLC",9,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation,7,,
1,Jan 10 2023 8:30AM,254596,10,"Nextsource Biotechnology, LLC",3,,
2,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc.",3,,
3,Jan 9 2023 4:27PM,254616,15,Lonza,1,,
4,Jan 9 2023 4:04PM,254613,15,"Person & Covey, Inc.",6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation,7.0,NaN,NaN
1,Jan 10 2023 8:30AM,254596,10,"Nextsource Biotechnology, LLC",3.0,NaN,NaN
2,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc.",3.0,NaN,NaN
3,Jan 9 2023 4:27PM,254616,15,Lonza,1.0,NaN,NaN
4,Jan 9 2023 4:04PM,254613,15,"Person & Covey, Inc.",6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 10 2023 8:30AM,254615,10,ISDIN Corporation,7.0,0.0,0.0
1,Jan 10 2023 8:30AM,254596,10,"Nextsource Biotechnology, LLC",3.0,0.0,0.0
2,Jan 9 2023 4:55PM,254617,15,"Person & Covey, Inc.",3.0,0.0,0.0
3,Jan 9 2023 4:27PM,254616,15,Lonza,1.0,0.0,0.0
4,Jan 9 2023 4:04PM,254613,15,"Person & Covey, Inc.",6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1782017,14.0,16.0,11.0
15,2036800,81.0,16.0,23.0
16,254585,1.0,0.0,0.0
19,1272958,14.0,0.0,0.0
20,509120,2.0,0.0,0.0
21,763794,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1782017,14.0,16.0,11.0
1,15,2036800,81.0,16.0,23.0
2,16,254585,1.0,0.0,0.0
3,19,1272958,14.0,0.0,0.0
4,20,509120,2.0,0.0,0.0
5,21,763794,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,14.0,16.0,11.0
1,15,81.0,16.0,23.0
2,16,1.0,0.0,0.0
3,19,14.0,0.0,0.0
4,20,2.0,0.0,0.0
5,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,14.0
1,15,Released,81.0
2,16,Released,1.0
3,19,Released,14.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,11.0,23.0,0.0,0.0,0.0,0.0
Executing,16.0,16.0,0.0,0.0,0.0,0.0
Released,14.0,81.0,1.0,14.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,11.0,23.0,0.0,0.0,0.0,0.0
1,Executing,16.0,16.0,0.0,0.0,0.0,0.0
2,Released,14.0,81.0,1.0,14.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,11.0,23.0,0.0,0.0,0.0,0.0
1,Executing,16.0,16.0,0.0,0.0,0.0,0.0
2,Released,14.0,81.0,1.0,14.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()